# LSSTCam pointing errors
## Looking at offsets more days.

Craig Lage - 18-Aug-25

In [ ]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION


In [ ]:
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()

## Get the data

In [ ]:
startDay = 20250806
endDay = 20250806

detector = camera['R22_S11']
bbox = detector.getBBox()

els = []
azs = []
point_ras = []
point_decs = []
true_ras = []
true_decs = []
deltaRas = []
deltaDecs = []

filters = []
pressures = []
temps = []
hums = []
times = []
dayObs = startDay
expIds = []
while dayObs <= endDay:
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", 
                where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
        exposureList.append([record.id, record])
    exposureList.sort(key=lambda x: x[0])
    print(len(exposureList))
    for [id,record] in exposureList:
        if record.observation_type not in ['acq', 'science']:
            continue
        try:
            calExp = butler.get('preliminary_visit_image', detector=94, visit=record.id, instrument=instrument)
            rawExp = butler.get('raw', detector=94, exposure=record.id, instrument=instrument)
            md = rawExp.getMetadata()
            cWcs = calExp.getWcs()
            if cWcs == None:
                print(f"{record.id} had no cWcs.")
                continue
            calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
            true_ra = calExpSkyCenter.getRa().asDegrees()
            true_dec = calExpSkyCenter.getDec().asDegrees()
            expIds.append(record.id)
            els.append((md['ELSTART'] + md['ELEND']) / 2.0)
            azs.append((md['AZSTART'] + md['AZEND']) / 2.0)
            filters.append(md['FILTBAND'])
            pressures.append(md['PRESSURE'])
            temps.append(md['AIRTEMP'])
            hums.append(md['HUMIDITY'])
            times.append((md['MJD-BEG'] + md['MJD-END']) / 2.0)
            true_ras.append(true_ra)
            true_decs.append(true_dec)
            point_ra = md['RASTART']
            point_dec = md['DECSTART']
            point_ras.append(point_ra)
            point_decs.append(point_dec)
            #print(ra, dec)
            deltaRa = (true_ra - point_ra) * 3600.0
            deltaDec = (true_dec - point_dec) * 3600.0
            deltaRas.append(deltaRa)
            deltaDecs.append(deltaDec)
            print(record.id, deltaRa, deltaDec)
        except:
            print(f"{record.id} failed!")
            continue
    print(dayObs, len(true_ras))
    dayObs = calcNextDay(dayObs)

filename = "/home/c/cslage/u/MTMount/mount_plots/pointing_results_06aug25.pkl"
with open(filename, 'wb') as f:
    pkl.dump([expIds, els, azs, true_ras, true_decs, point_ras, point_decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters], f)


In [ ]:
filename = "/home/c/cslage/u/MTMount/mount_plots/pointing_results_06aug25.pkl"
with open(filename, 'rb') as f:
    [expIds, els, azs, true_ras, true_decs, point_ras, point_decs, deltaRas, deltaDecs, pressures, temps, hums, times, filters] = pkl.load(f)
len(els)

In [ ]:
dayObs = 20250806
separation = []
for i in range(len(point_ras)):
    sep = np.sqrt(((deltaRas[i] * np.cos(point_decs[i] * np.pi / 180.0))**2 + deltaDecs[i]**2))
    separation.append(sep)
        
plt.hist(separation, bins = 50, range=(0,100), alpha=0.5)

plt.xlim(0, 100)
plt.legend()
plt.title(f"On sky pointing model error, {dayObs}")
plt.xlabel("Error (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/OnSky_Pointing_Errors_{dayObs}.png")

In [ ]:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}

deltaAzs = []
deltaEls = []
point_azs = []
point_els = []
true_azs = []
true_els = []
for index in range(len(true_ras)):
    skyLocation = SkyCoord(true_ras[index]*u.deg, true_decs[index]*u.deg)
    time = Time(times[index], format='mjd', scale='tai')
    pressure = pressures[index] * u.pascal
    temperature = temps[index] * u.Celsius
    hum = hums[index]
    wl = wavelengths[filters[index]] * u.angstrom
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz = skyLocation.transform_to(altAz)
    true_az = Angle(obsAltAz.az.deg * u.deg)
    wrapped_true_az = true_az.wrap_at(360.0 * u.deg)
    true_azs.append(wrapped_true_az.deg)
    az = Angle(azs[index] * u.deg)
    wrapped_az = az.wrap_at(360.0 * u.deg)
    deltaAz = az.deg - true_az.deg

    if deltaAz > 360.0:
        deltaAz -= 360.0
    if deltaAz < -180.0:
        deltaAz += 360.0

    deltaAz *= 3600.0
    true_el = obsAltAz.alt.deg
    deltaEl = (els[index] - true_el) * 3600.0
    print(expIds[index], deltaAz, deltaEl)
    print(expIds[index], wrapped_az.deg, wrapped_true_az.deg)
    
    true_els.append(true_el)
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
    point_azs.append(wrapped_az.deg)
    point_els.append(els[index])
    
len(point_els)        

In [ ]:
print(len(deltaEls), len(deltaRas))

In [ ]:
index

In [ ]:
deltaDecs[index]

In [ ]:
print(deltaAzs[index], deltaEls[index])

In [ ]:
time.isot

In [ ]:
time.utc.isot

In [ ]:
# From RUbinTV
#2025-08-07T09:28:40.200326

In [ ]:
skyLocation = SkyCoord(point_ras[index]*u.deg, point_decs[index]*u.deg)
time = Time(times[index], format='mjd', scale='tai')
pressure = pressures[index] * u.pascal
temperature = temps[index] * u.Celsius
hum = hums[index]
wl = wavelengths[filters[index]] * u.angstrom
altAz = AltAz(obstime=time, location=SIMONYI_LOCATION, pressure=pressure, 
             temperature=temperature, relative_humidity=hum, obswl=wl)
obsAltAz = skyLocation.transform_to(altAz)


In [ ]:
print(point_azs[index], obsAltAz.az.deg, (point_azs[index] - obsAltAz.az.deg)*3600.0)

In [ ]:

plt.figure(figsize=(8,8))
plt.suptitle("LSST Delta AltAz 2025-08-06")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(point_els, deltaEls, c=point_azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(point_azs, deltaEls,c=point_els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(point_els, deltaAzs, c=point_azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(0,90)
plt.ylabel('Delta Az arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(point_azs, deltaAzs,c=point_els, cmap=plt.cm.coolwarm)
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Delta Az arcsec')
plt.savefig("/home/c/cslage/u/MTMount/mount_plots/Delta_AltAz_06Aug25.png")


In [ ]:
dayObs = 20250806
separation = []
for i in range(len(point_azs)):
    sep = np.sqrt(((deltaAzs[i] * np.cos(point_els[i] * np.pi / 180.0))**2 + deltaEls[i]**2))
    separation.append(sep)
        
plt.hist(separation, bins = 50, range=(0,300), alpha=0.5)

plt.xlim(0, 300)
plt.legend()
plt.title(f"AzEl pointing model error, {dayObs}")
plt.xlabel("Error (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/AzEl_Pointing_Errors_{dayObs}.png")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(np.array(point_azs) * np.pi / 180.0, 90.0 - np.array(point_els), marker = 'x')
ax.set_rmax(60.0)
r_values = [10.0, 20.0, 30.0, 40.0, 50.0]
r_labels = [80.0, 70.0, 60.0, 50.0, 40.0]
ax.set_rgrids(r_values, r_labels)
ax.grid(True)
plt.savefig("/home/c/cslage/u/LSSTCam/data/Sky_Coverage_2025-08-06.png")

# Generate a tpoint input file with these errors

In [ ]:
outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_Fixed_06Aug25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,Aug 6, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")

for i in range(len(point_els)):
    outfile.write(f"{true_azs[i]:.9f}\t{true_els[i]:.6f}\t{point_azs[i]:.9f}\t{point_els[i]:.9f}\n")
outfile.write("END\n")
outfile.close()

